In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config("Users/odl_user_281290/config.json")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

StatementMeta(b6712890-fba9-472d-a032-aa60f0c8e41e, 3, 6, Finished, Available, Finished)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CCRNQ35MW to authenticate.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-281290
Azure region: westeurope
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-281290


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "project-1-cluster-2"

# DONE: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing compute target: ", cluster_name)
except ComputeTargetException:
    print("Creating a new compute target: ", cluster_name)
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", min_nodes=0, max_nodes=6)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

StatementMeta(b6712890-fba9-472d-a032-aa60f0c8e41e, 3, 7, Finished, Available, Finished)

Found existing compute target:  project-1-cluster-2
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
!pip install azureml-sdk
!pip install azureml.widgets

StatementMeta(b6712890-fba9-472d-a032-aa60f0c8e41e, 3, 11, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 98.1 MB/s eta 0:00:0000:0100:01


In [23]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.01, 0.1, 1, 5),
    '--max_iter': choice(50, 100, 150, 250)
}) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(
    slack_factor=0.1,
    evaluation_interval=2,
) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='Users/odl_user_281290/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='Users/odl_user_281290/',
    script='train.py',
    compute_target=compute_target,
    environment=sklearn_env
) ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=5
) ### YOUR CODE HERE ###

StatementMeta(b6712890-fba9-472d-a032-aa60f0c8e41e, 3, 28, Finished, Available, Finished)

In [24]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hd_run = exp.submit(config=hyperdrive_config)
RunDetails(hd_run).show()
hd_run.wait_for_completion(show_output=True)

StatementMeta(b6712890-fba9-472d-a032-aa60f0c8e41e, 3, 29, Finished, Available, Finished)

FileNotFoundError: [Errno 2] No such file or directory: 'Users/odl_user_281290/__pycache__/train.cpython-310.pyc'

In [14]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hd_run.get_best_run_by_primary_metric()
model = best_run.register_model(model_name='best_model', model_path='Users/odl_user_281290/outputs/model.joblib')
joblib.dump(model, 'Users/odl_user_281290/outputs/best_model.joblib')
print('Best run ID: ', best_run.id)
print("Parameters: ", best_run.get_details()['runDefinition']['arguments'])

StatementMeta(b6712890-fba9-472d-a032-aa60f0c8e41e, 3, 19, Finished, Available, Finished)

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path Users/odl_user_281290/outputs/model.joblib in the set of files uploaded to the run: ['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2025-05-24-11', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path Users/odl_user_281290/outputs/model.joblib in the set of files uploaded to the run: ['logs/azureml/dataprep

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

In [ ]:
from Users.odl_user_281290.train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
training_data = pd.concat([x, y], axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=training_data,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, fitted_model = automl_run.get_output()
joblib.dumb(fitted_model, 'Users/odl_user_281290/outputs/best_automl_model.pkl')
print('Best model saved successfully!')

In [ ]:
compute_target.delete()